In [1]:


! curl -L https://github.com/towhee-io/examples/releases/download/data/reverse_image_search.zip -O
! unzip -q -o reverse_image_search.zip



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  119M  100  119M    0     0  6118k      0  0:00:20  0:00:20 --:--:-- 6324k


In [1]:
from PIL import Image
import pandas as pd
import torch
from transformers import CLIPProcessor, CLIPModel
import numpy as np


# Load the dataset
dataset_path = 'reverse_image_search.csv'  # Replace with your dataset path
df = pd.read_csv(dataset_path)

# Load the CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")




/home/kasim/anaconda3/envs/milvus/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialize a list to store tensors
embeddings = []

for index, row in df.iterrows():
    image_path = row['path']  # Assuming the path is in a column named 'path'
    image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB
    inputs = processor(images=image, return_tensors="pt")
    image_features = model.get_image_features(**inputs)
    # Ensure the tensor is detached from the computational graph before converting
    embedding = image_features.squeeze(0).detach().numpy().tolist()
    embedding = embedding/np.linalg.norm(embedding)

    embeddings.append(embedding)

# Concatenate all feature vectors into a single tensor
#image_features_tensor = torch.stack(embeddings)

# image_features_tensor now contains the feature vectors for all images in your dataset>

In [3]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

# Milvus parameters
HOST = '127.0.0.1'
PORT = '19530'
TOPK = 13

In [4]:
def create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    
    fields = [
        FieldSchema(name='path', dtype=DataType.VARCHAR, description='path to image', max_length=500, 
                    is_primary=True, auto_id=False),
        FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, description='image embedding vectors', dim=dim)
    ]
    schema = CollectionSchema(fields=fields, description='reverse image search')
    collection = Collection(name=collection_name, schema=schema)

    index_params = {
        'metric_type': METRIC_TYPE,
        'index_type': INDEX_TYPE,
        'params': {"nlist": 512}
    }
    collection.create_index(field_name='embedding', index_params=index_params)
    return collection

In [5]:
connections.connect(host=HOST, port=PORT)
collection_name = 'eucledian_collection'
dim = 512  # Dimension of the embeddings
METRIC_TYPE = 'L2'  # You can choose 'L2', 'IP', etc., based on your requirement
INDEX_TYPE = 'IVF_FLAT'  # Index type

In [6]:
collection = create_milvus_collection(collection_name, dim)
paths = df['path'].tolist()
entities = [[path for path in paths],
            [embedding for embedding in embeddings]]
mr = collection.insert(entities)


In [7]:
connections.connect(host=HOST, port=PORT)
collection_name = 'cos_collection'
dim = 512  # Dimension of the embeddings
METRIC_TYPE = 'COSINE'  # You can choose 'L2', 'IP', etc., based on your requirement
INDEX_TYPE = 'IVF_PQ'  # Index type

In [9]:
collection = create_milvus_collection(collection_name, dim)
paths = df['path'].tolist()
entities = [[path for path in paths],
            [embedding for embedding in embeddings]]
mr = collection.insert(entities)


RPC error: [create_index], <MilvusException: (code=65535, message=parameter `m` not found)>, <Time:{'RPC start': '2024-05-14 23:30:59.340542', 'RPC error': '2024-05-14 23:30:59.344843'}>


MilvusException: <MilvusException: (code=65535, message=parameter `m` not found)>

In [10]:
connections.connect(host=HOST, port=PORT)
collection_name = 'inner_product_collection'
dim = 512  # Dimension of the embeddings
METRIC_TYPE = 'IP'  # You can choose 'L2', 'IP', etc., based on your requirement
INDEX_TYPE = 'IVF_FLAT'  # Index type

In [11]:
collection = create_milvus_collection(collection_name, dim)
paths = df['path'].tolist()
entities = [[path for path in paths],
            [embedding for embedding in embeddings]]
mr = collection.insert(entities)


In [12]:
utility.list_collections()

['tranformers_clip_patch16',
 'eucledian_collection',
 'cos_collection',
 'text_image_search',
 'default_collection',
 'default',
 'inner_product_collection']

In [19]:
collection = Collection(name=collection_name)

In [16]:
#eucledian
collection = Collection(name="eucledian_collection")
collection.load()

In [17]:
search_params = {
    "metric_type": "L2", 
    "offset": 0, 
    "ignore_growing": False, 
    "params": {"nprobe": 10}
}

In [18]:
# search with text 
#patch 16
query_text = "airplane"  
text_inputs = processor(text=query_text, return_tensors="pt")
query_text_features = model.get_text_features(**text_inputs)
text_embedding = query_text_features.squeeze(0).detach().numpy().tolist()

text_embedding = text_embedding/np.linalg.norm(text_embedding)



In [19]:
results = collection.search(
    data=[text_embedding], 
    anns_field="embedding", 
    # the sum of `offset` in `param` and `limit` 
    # should be less than 16384.
    param=search_params,
    limit=10,
    expr=None,
)



In [20]:
results[0].ids

['./train/warplane/n04552348_16150.JPEG',
 './train/parachute/n03888257_4738.JPEG',
 './train/warplane/n04552348_13334.JPEG',
 './train/warplane/n04552348_12780.JPEG',
 './train/parachute/n03888257_25150.JPEG',
 './train/horizontal_bar/n03535780_40969.JPEG',
 './train/can_opener/n02951585_32431.JPEG',
 './train/bullet_train/n02917067_12974.JPEG',
 './train/parachute/n03888257_13175.JPEG',
 './train/can_opener/n02951585_24077.JPEG']

In [28]:
results[0].distances

[0.2719953656196594,
 0.24478015303611755,
 0.24317307770252228,
 0.23824922740459442,
 0.23627306520938873,
 0.23189441859722137,
 0.22985076904296875,
 0.22882045805454254,
 0.22614918649196625,
 0.2248741090297699]

In [22]:
#eucledian
collection = Collection(name="inner_product_collection")
collection.load()

In [23]:
search_params = {
    "metric_type": "IP", 
    "offset": 0, 
    "ignore_growing": False, 
    "params": {"nprobe": 10}
}

In [24]:
# search with text 
#patch 16
query_text = "airplane"  
text_inputs = processor(text=query_text, return_tensors="pt")
query_text_features = model.get_text_features(**text_inputs)
text_embedding = query_text_features.squeeze(0).detach().numpy().tolist()

text_embedding = text_embedding/np.linalg.norm(text_embedding)



In [26]:
results[0].ids

['./train/warplane/n04552348_16150.JPEG',
 './train/parachute/n03888257_4738.JPEG',
 './train/warplane/n04552348_13334.JPEG',
 './train/warplane/n04552348_12780.JPEG',
 './train/parachute/n03888257_25150.JPEG',
 './train/horizontal_bar/n03535780_40969.JPEG',
 './train/can_opener/n02951585_32431.JPEG',
 './train/bullet_train/n02917067_12974.JPEG',
 './train/parachute/n03888257_13175.JPEG',
 './train/can_opener/n02951585_24077.JPEG']

In [27]:
results[0].distances

[0.2719953656196594,
 0.24478015303611755,
 0.24317307770252228,
 0.23824922740459442,
 0.23627306520938873,
 0.23189441859722137,
 0.22985076904296875,
 0.22882045805454254,
 0.22614918649196625,
 0.2248741090297699]